# 1.1

In [1]:
%matplotlib qt
import os
import matplotlib.pyplot as plt
import imageio
import numpy as np

# Load the video and grab the first frame
reader = imageio.get_reader('ar5.gif')
first_frame = reader.get_data(0)

if os.path.exists('points_2d_initial.npy'):
    points_2d = np.load('points_2d_initial.npy')
else:
    plt.imshow(first_frame)
    plt.title("Click on corners")
    points_2d = plt.ginput(30, timeout=0) 
    plt.close()

    points_2d = np.array(points_2d)
    np.save('points_2d_initial.npy', points_2d)

points_3d = np.array([
    [0, 0, 0], [1, 0, 0], [2, 0, 0], [3, 0, 0], [4, 0, 0],
    [0, 0, 1], [1, 0, 1], [2, 0, 1], [3, 0, 1], [4, 0, 1],
    [0, 0, 2], [1, 0, 2], [2, 0, 2], [3, 0, 2], [4, 0, 2],
    [0, 1, 2], [1, 1, 2], [2, 1, 2], [3, 1, 2], [4, 1, 2],
    [0, 2, 2], [1, 2, 2], [2, 2, 2], [3, 2, 2], [4, 2, 2],
    [0, 3, 2], [1, 3, 2], [2, 3, 2], [3, 3, 2], [4, 3, 2]])

In [2]:
import cv2

trackers = []
for pt in points_2d:
    tracker = cv2.legacy.TrackerMedianFlow_create()
    
    # 8x8 bounding box
    bbox = (pt[0] - 4, pt[1] - 4, 8, 8) 
    tracker.init(first_frame, bbox)
    trackers.append(tracker)

# 1.2

In [ ]:
import imageio
import cv2
import numpy as np

reader = imageio.get_reader('ar5.gif') 
writer = imageio.get_writer('tracking_visualization.mp4', fps=10, format='FFMPEG', mode='I')

for frame in reader:
    vis_frame = frame.copy()
    
    for i, tracker in enumerate(trackers):
        # Update the tracker
        success, bbox = tracker.update(frame)
        
        if success:
            x, y, w, h = [int(v) for v in bbox]
            center = (x + w//2, y + h//2)

            # Draw the green dot
            cv2.circle(vis_frame, center, 3, (0, 255, 0), -1) 
            
    writer.append_data(vis_frame)

writer.close()
reader.close()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (190, 336) to (192, 336) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
